In [13]:
# !pip install scikit-learn xgboost

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error


df = pd.read_csv("data/train.csv")

In [14]:
# print(df["신고접수번호"].value_counts())
# print(df["접수경로"].value_counts())
# print(df["신고접수일시"].value_counts())
# print(df["시군구"].value_counts())
# print(df["접수분류"].value_counts().sum())
# print(df["긴급구조종류"].value_counts())
print(df["접수분류"].isnull().sum())

0


In [15]:
df["dt"] = df["신고접수일시"].apply(lambda x : datetime.strptime(x, "%Y%m%d_%H%M"))
df["_dt"] = df["dt"] + timedelta(hours=6)
df["_date"] = df._dt.dt.date.astype(str)
df["_hour"] = df._dt.dt.hour
df


,신고접수번호,접수경로,신고접수일시,시군구,접수분류,긴급구조종류,dt,_dt,_date,_hour
0,JGT2IBW4,이동전화,20130101_0001,NaN,안내,NaN,2013-01-01 00:01:00,2013-01-01 06:01:00,2013-01-01,6
1,74ED11Z4,기타,20130101_0002,NaN,안내,NaN,2013-01-01 00:02:00,2013-01-01 06:02:00,2013-01-01,6
2,B4I8RIBW,이동전화,20130101_0002,NaN,안내,NaN,2013-01-01 00:02:00,2013-01-01 06:02:00,2013-01-01,6
3,482FI3AJ,이동전화,20130101_0003,북구,출동,구급,2013-01-01 00:03:00,2013-01-01 06:03:00,2013-01-01,6
4,AR9N3QT4,이동전화,20130101_0004,NaN,안내,NaN,2013-01-01 00:04:00,2013-01-01 06:04:00,2013-01-01,6
...,...,...,...,...,...,...,...,...,...,...
6415797,KAOUMKLU,기타,20220630_2354,NaN,안내,NaN,2022-06-30 23:54:00,2022-07-01 05:54:00,2022-07-01,5
6415798,LAI1MQ2K,이동전화,20220630_2354,사상구,출동,구급,2022-06-30 23:54:00,2022-07-01 05:54:00,2022-07-01,5
6415799,N5KF2ELA,이동전화,20220630_2356,동구,출동,구급,2022-06-30 23:56:00,2022-07-01 05:56:00,2022-07-01,5
6415800,53Q98D6Z,의료지도연결,20220630_2358,NaN,안내,NaN,2022-06-30 23:58:00,2022-07-01 05:58:00,2022-07-01,5


In [16]:
df_ = df.pivot_table(index = ['_date', '_hour'], columns = '접수분류', aggfunc='size').reset_index()
print(df_.describe())

접수분류          _hour            안내            출동
count  83232.000000  83229.000000  83232.000000
mean      11.500000     48.236384     28.848712
std        6.922228     45.468295     13.951314
min        0.000000      1.000000      1.000000
25%        5.750000     26.000000     19.000000
50%       11.500000     41.000000     28.000000
75%       17.250000     60.000000     37.000000
max       23.000000   2974.000000    285.000000


In [17]:
print("\n==============\n")

# 야간 시간별 안내, 출동
# 주간 시간별 안내, 출동

night_date = df_[(df_["_hour"] >= 0) & (df_["_hour"] < 15)].groupby("_date").sum()
night_date = night_date.reset_index()
night_date.drop(["_hour"], axis=1, inplace=True)
night_date.columns = ["date", "night_0", "night_1"]
print(night_date)

day_date = df_[(df_["_hour"] >= 15) & (df_["_hour"] < 24)].groupby("_date").sum()
day_date = day_date.reset_index()
day_date.drop(["_hour"], axis=1, inplace=True)
day_date.columns = ["date", "day_0", "day_1"]
print(day_date)



            date  night_0  night_1
0     2013-01-01    406.0    207.0
1     2013-01-02    491.0    294.0
2     2013-01-03    766.0    247.0
3     2013-01-04    781.0    224.0
4     2013-01-05    467.0    324.0
...          ...      ...      ...
3464  2022-06-27    782.0    593.0
3465  2022-06-28    608.0    422.0
3466  2022-06-29    684.0    475.0
3467  2022-06-30    591.0    467.0
3468  2022-07-01    437.0    271.0

[3469 rows x 3 columns]
            date   day_0  day_1
0     2013-01-01  1851.0  206.0
1     2013-01-02   289.0  272.0
2     2013-01-03   316.0  261.0
3     2013-01-04   328.0  294.0
4     2013-01-05   492.0  294.0
...          ...     ...    ...
3463  2022-06-26   670.0  359.0
3464  2022-06-27   682.0  417.0
3465  2022-06-28   499.0  385.0
3466  2022-06-29   471.0  356.0
3467  2022-06-30   529.0  372.0

[3468 rows x 3 columns]


In [21]:
data = pd.concat([night_date, day_date], axis="columns")
data = pd.merge(left=night_date, right=day_date, on="date")
data["day_y"] = data["day_0"] + data["day_1"]
data.drop(["day_0", "day_1"], axis=1, inplace=True)
data

,date,night_0,night_1,day_y
0,2013-01-01,406.0,207.0,2057.0
1,2013-01-02,491.0,294.0,561.0
2,2013-01-03,766.0,247.0,577.0
3,2013-01-04,781.0,224.0,622.0
4,2013-01-05,467.0,324.0,786.0
...,...,...,...,...
3463,2022-06-26,865.0,491.0,1029.0
3464,2022-06-27,782.0,593.0,1099.0
3465,2022-06-28,608.0,422.0,884.0
3466,2022-06-29,684.0,475.0,827.0


In [19]:
train, val = train_test_split(data, train_size=0.8, shuffle=False)
X_columns = ["night_0", "night_1"]
y_columns = ["day_y"]

X_train = train[X_columns].to_numpy()
y_train = train[y_columns].to_numpy()
X_val = val[X_columns].to_numpy()
y_val = val[y_columns].to_numpy()

xgb_params = {
    "n_estimator" : 100,
    "max_depth" : 5,
    "eval_metric" : "mape",
    "early_stopping_rounds" : 10,
    "random_state": 514
}

model = XGBRegressor(**xgb_params)
eval_set = [(X_val, y_val)]

model.fit(X=X_train,
          y=y_train,
          eval_set=eval_set,
          verbose=True)
prediction = model.predict(X_val)

print(mean_absolute_percentage_error(y_val, prediction))



[0]	validation_0-mape:0.15585


[1]	validation_0-mape:0.14750
[2]	validation_0-mape:0.15609
[3]	validation_0-mape:0.16342
[4]	validation_0-mape:0.17046
[5]	validation_0-mape:0.17175
[6]	validation_0-mape:0.17377
[7]	validation_0-mape:0.17565
[8]	validation_0-mape:0.17723
[9]	validation_0-mape:0.17902
[10]	validation_0-mape:0.17972
[11]	validation_0-mape:0.17973
0.1475047621991673


/opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:34:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


In [20]:

test_df = pd.read_csv("data/test.csv")
test_df["dt"] = test_df["신고접수일시"].apply(lambda x : datetime.strptime(x, "%Y%m%d_%H%M"))
test_df["_dt"] = test_df["dt"] + timedelta(hours=6)
test_df["_date"] = test_df._dt.dt.date.astype(str)
test_df["_hour"] = test_df._dt.dt.hour
test_df_ = test_df.pivot_table(index = ['_date', '_hour'], columns = '접수분류', aggfunc='size').reset_index()
target_df = test_df_[(test_df_["_hour"] >= 0) & (test_df_["_hour"] < 15)].groupby("_date").sum()
target_df = target_df.reset_index()
target_df.drop(["_hour"], axis=1, inplace=True)
target_df.columns = ["date", "night_0", "night_1"]

target_df

test_preds = model.predict(target_df[X_columns])

submission = pd.read_csv('data/sample_submission.csv')
submission["y"] = test_preds.reshape(-1)
submission.to_csv("submission.csv", index=False)

# import pickle

# # insert model_name
# model_name = "02" # <------
# ## Save pickle
# with open(f"{model_name}.pickle","wb") as f:
#     pickle.dump(model, f)

# # Load pickle
# with open(f"{model_name}.pickle","rb") as f:
#     data = pickle.load(f)
#     print(data)
